In [ ]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import time
sns.set()


titles = []
prices = []
areas = []
zone = []
descriptions = []



n_pages = 0

headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

for page in range(0,10):
    n_pages += 1
    URL = 'https://casa.sapo.pt/Venda/Apartamentos/?sa=11&lp=10000&or=10'+'&pn='+str(page)
    response = get(URL, headers=headers)
    print(response)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    house_containers = html_soup.find_all('div',class_="searchResultProperty item")

    if house_containers != []:
        for container in house_containers:

            price = container.find_all('span')[2].text

            if price == 'Contacte Anunciante':
                price = container.find_all('span')[3].text
                if price.find('/') != -1:
                    price = price[0:price.find('/')-1]
            if price.find('/') != -1:
                price = price[0:price.find('/')-1]

            price_ = [int(price[s]) for s in range(0,len(price)) if price[s].isdigit()]
            price = ''
            for x in price_:
                price = price+str(x)
            prices.append(int(price))


            #location
            location=container.find_all('p',class_="searchPropertyLocation")[0].text
            location1=location[7:location.find(',')]
            print(location1)
            zone.append(location1)

            #title
            name = container.find_all('span')[0].text
            titles.append(name)

            # Area
            m2 = container.find_all('p')[9].text
            if m2 != '-':
                m2 = m2.replace('\xa0','')
                m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                areas.append(m2)

            else:
                m2 = container.find_all('p')[7].text
                if m2 != '-':
                    m2 = m2.replace('\xa0','')
                    m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                    areas.append(m2)
                else:
                    areas.append(m2)


            #Descriptions

            SD = container.find_all('p',class_="searchPropertyDescription")[0].text[7:-6]
            descriptions.append(SD)
    else:
        break

#     Sleep(randint(1,2))

print('You scraped {} pages containing {} properties.'.format(n_pages, len(titles)))
print("hi ")


cols = ['Price','Zone','Title','Size (m²)','Description',]

lisboa = pd.DataFrame({    'Price': prices,
                           'Zone': zone,
                           'Title': titles,
                           'Size (m²)': areas,
                           'Description': descriptions,
                            })[cols]

lisboa.to_excel('data_raw.xls','w')

